In [4]:
import os
import numpy as np
import zipfile
from ultralytics import YOLO
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms
import cv2 as cv
import pandas as pd
import matplotlib.pyplot as plt

In [12]:
# if it is unzipped, unzip kagglecatsanddogs_5340.zip
if not os.path.exists('PetImages'):
    # unzip kagglecatsanddogs_5340.zip
    print("The file is not unzipped, unzipping it now")
    with zipfile.ZipFile("kagglecatsanddogs_5340.zip","r") as zip_ref:
        zip_ref.extractall()
    # delete unnecessary files
    os.remove("CDLA-Permissive-2.0.pdf")
    os.remove("readme[1].txt")
else:
    print("The file is already unzipped")

The file is not unzipped, unzipping it now


In [25]:
myDf = pd.DataFrame(columns=["A", "B"])
# add a row to the dataframe
myDf.loc[0] = [1, 2]
print(myDf.head())

   A  B
0  1  2


In [26]:
# rename the images as img_0.jpg, img_1.jpg, etc. and save the labels in a pandas dataframe where 0 is cat and 1 is dog
# there is ID and label in the dataframe
# the labels are 0 for cat and 1 for dog
# images folder should be in the same directory as this file

if not os.path.exists('images'):
    print("Creating images folder and labels dataframe")
    os.makedirs('images')
    labels = pd.DataFrame(columns=['ID', 'label'])
    count = 0
    for folder in ['Cat', 'Dog']:
        for i, file in enumerate(os.listdir('PetImages/'+folder)):
            try:
                img_array = cv.imread('PetImages/'+folder+'/'+file)
                os.rename('PetImages/'+folder+'/'+file, 'images/img_'+str(count)+'.jpg')
                labels.loc[count] = [count, 0 if folder == 'Cat' else 1]
                count += 1
            except Exception as e:
                print(e)
                print('Deleting ', 'PetImages/'+folder+'/'+file)
                os.remove('PetImages/'+folder+'/'+file)
    # delete the PetImages folder
    if os.path.exists('PetImages'):
        if os.path.exists('PetImages/Cat'):
            os.rmdir('PetImages/Cat')
        if os.path.exists('PetImages/Dog'):
            os.rmdir('PetImages/Dog')
        os.rmdir('PetImages')
    #  save the labels dataframe as a csv file
    labels.to_csv('labels.csv')
else:
    print("images folder already exists")

        

Creating images folder and labels dataframe
OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgcodecs\src\loadsave.cpp:816: error: (-215:Assertion failed) !buf.empty() in function 'cv::imdecode_'

Deleting  PetImages/Cat/666.jpg
OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgcodecs\src\loadsave.cpp:816: error: (-215:Assertion failed) !buf.empty() in function 'cv::imdecode_'

Deleting  PetImages/Dog/11702.jpg


In [ ]:
#defining dataset class
class Dataset(torch.utils.data.Dataset):
      'Characterizes a dataset for PyTorch'
      def __init__(self, image_dir, label_dir, transform=None):
            'Initialization'
            self.image_dir = image_dir
            self.label_dir = label_dir
            self.transform = transform

            self.labels = pd.read_csv(label_dir)
            self.processed_images = self.process_images(self.image_dir)

      def process_images(self, image_dir):
            processed_images = []
            for img in os.listdir(image_dir):
                  img_array = cv.imread(image_dir + '/' + img)
                  new_array = cv.resize(img_array, (256, 256))
                  img = transforms.ToTensor()(img)
                  img = transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))(img) # Normalize the tensor values to the range (-1, 1)
                  processed_images.append(new_array)
            return processed_images


      def __len__(self):
            'Denotes the total number of images'
            return len(self.process_images)

      def __getitem__(self, index):
            'Generates one sample of data'
            # Select sample
            ID = self.list_IDs[index]

            # Load data and get label
            X = torch.load('data/' + ID + '.pt')
            y = self.labels[ID]

            return X, y
